In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
data = pd.read_csv('train_processed.csv')
data['issue_date'] = pd.to_datetime(data['issue_date'])
data['month'] = data['issue_date'].dt.month
data['year'] = data['issue_date'].dt.year
cols = ['usgs_rw','mnf_rw','mjo_rw','nino_rw','oni_rw','pdo_rw','pna_rw','soi_rw']
def string_to_float_list(s):
    items = s.strip('[]').split(',')
    return [float(item.strip()) for item in items if item]
for col in cols:
    data[col] = data[col].apply(string_to_float_list)
data

,site_id,year,volume,issue_date,volume_scaled,usgs_rw,mnf_scaled,mnf_rw,pred_mar,pred_apr,pred_may,pred_jun,pred_jul,mjo_rw,nino_rw,oni_rw,pdo_rw,pna_rw,soi_rw,month
0,american_river_folsom_lake,1990,532.100,1990-01-01,0.085628,"[0.0028622252420492066, 0.002893983947030845, ...",0.097819,"[0.03172846549634365, 0.04688166447469881, 0.0...",0.114502,0.125828,0.084614,0.082711,0.031674,"[-0.5, -0.38, 0.47, 0.73, 1.32, -0.03, -0.64, ...","[-0.48, -0.23, 0.0, -0.13, -1.25, -1.15, -0.7,...","[-1.85, -1.69, -1.43, -1.08, -0.83, -0.58, -0....","[-1.24, -1.45, -1.24, -0.32, -0.09, 0.11, 0.56...","[-0.72, -1.06, -1.3, -0.54, -0.14, -0.63, -0.1...","[1.5, 1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5,...",1
1,american_river_folsom_lake,1990,532.100,1990-01-08,0.085628,"[0.0027386981453826754, 0.0026116633254561206,...",0.097819,"[0.03172846549634365, 0.04688166447469881, 0.0...",0.114502,0.125828,0.084614,0.082711,0.031674,"[0.47, 0.73, 1.32, -0.03, -0.64, -1.22, -0.63,...","[-0.48, -0.23, 0.0, -0.13, -1.25, -1.15, -0.7,...","[-1.85, -1.69, -1.43, -1.08, -0.83, -0.58, -0....","[-1.24, -1.45, -1.24, -0.32, -0.09, 0.11, 0.56...","[-0.72, -1.06, -1.3, -0.54, -0.14, -0.63, -0.1...","[1.5, 1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5,...",1
2,american_river_folsom_lake,1990,532.100,1990-01-15,0.085628,"[0.0022214959691747344, 0.0021420992067206373,...",0.097819,"[0.03172846549634365, 0.04688166447469881, 0.0...",0.114502,0.125828,0.084614,0.082711,0.031674,"[0.73, 1.32, -0.03, -0.64, -1.22, -0.63, 0.46,...","[-0.48, -0.23, 0.0, -0.13, -1.25, -1.15, -0.7,...","[-1.85, -1.69, -1.43, -1.08, -0.83, -0.58, -0....","[-1.24, -1.45, -1.24, -0.32, -0.09, 0.11, 0.56...","[-0.72, -1.06, -1.3, -0.54, -0.14, -0.63, -0.1...","[1.5, 1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5,...",1
3,american_river_folsom_lake,1990,532.100,1990-01-22,0.085628,"[0.0018314828817957956, 0.0018156035293049762,...",0.097819,"[0.03172846549634365, 0.04688166447469881, 0.0...",0.114502,0.125828,0.084614,0.082711,0.031674,"[1.32, -0.03, -0.64, -1.22, -0.63, 0.46, 0.56,...","[-0.48, -0.23, 0.0, -0.13, -1.25, -1.15, -0.7,...","[-1.85, -1.69, -1.43, -1.08, -0.83, -0.58, -0....","[-1.24, -1.45, -1.24, -0.32, -0.09, 0.11, 0.56...","[-0.72, -1.06, -1.3, -0.54, -0.14, -0.63, -0.1...","[1.5, 1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5,...",1
4,american_river_folsom_lake,1990,532.100,1990-02-01,0.085628,"[0.0015774132419426855, 0.0015003400544524373,...",0.061772,"[0.04688166447469881, 0.057554633545957676, 0....",0.114502,0.125828,0.084614,0.082711,0.031674,"[-0.64, -1.22, -0.63, 0.46, 0.56, -0.31, -0.87...","[-0.23, 0.0, -0.13, -1.25, -1.15, -0.7, -0.51,...","[-1.69, -1.43, -1.08, -0.83, -0.58, -0.4, -0.3...","[-1.45, -1.24, -0.32, -0.09, 0.11, 0.56, -0.44...","[-1.06, -1.3, -0.54, -0.14, -0.63, -0.18, -0.2...","[1.2, 1.1, 1.6, 1.2, 0.7, 0.9, -0.3, 0.5, 0.8,...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17467,yampa_r_nr_maybell,2022,680.923,2022-06-22,0.325856,"[0.2350909063075659, 0.19937133555475045, 0.17...",0.155663,"[0.017243066692590858, 0.05901254930924646, 0....",0.017243,0.059013,0.204278,0.155663,0.025318,"[0.74, 0.26, -0.84, 0.42, 0.34, -0.87, -0.7, -...","[-0.05, 0.25, -0.01, 0.18, -0.29, -0.81, -1.53...","[-0.48, -0.38, -0.4, -0.49, -0.67, -0.81, -0.9...","[-1.81, -1.96, -0.94, -1.96, -3.11, -2.75, -2....","[0.67, 0.56, 0.95, 0.44, 1.13, 0.72, -2.56, 1....","[0.4, 1.4, 0.6, 0.8, 0.7, 1.0, 1.5, 0.5, 1.1, ...",6
17468,yampa_r_nr_maybell,2022,680.923,2022-07-01,0.325856,"[0.22335447591735513, 0.19222742140418736, 0.1...",0.025318,"[0.05901254930924646, 0.20427827444061816, 0.1...",0.017243,0.059013,0.204278,0.155663,0.025318,"[0.26, -0.84, 0.42, 0.34, -0.87, -0.7, -0.61, ...","[0.25, -0.01, 0.18, -0.29, -0.81, -1.53, -0.7,...","[-0.38, -0.4, -0.49, -0.67, -0.81, -0.98, -0.9...","[-1.96, -0.94, -1.96, -3.11, -2.75, -2.71, -2....","[0.56, 0.95, 0.44, 1.13, 0.72, -2.56, 1.01, 0....","[1.4, 0.6, 0.8, 0.7, 1.0, 1.5, 0.5, 1.1, 1.8, ...",7
17469,yampa_r_nr_may

In [35]:
from geopy.distance import geodesic
meta = pd.read_csv('./data/metadata.csv')
locations = meta.set_index('site_id')[['latitude', 'longitude']].T.apply(tuple).to_dict()

def calculate_distances(locations):
    distances = {site: {} for site in locations}
    for site1, coords1 in locations.items():
        for site2, coords2 in locations.items():
            if site1 != site2:
                lat_long_1 = (coords1['latitude'], coords1['longitude'])
                lat_long_2 = (coords2['latitude'], coords2['longitude'])
                distances[site1][site2] = geodesic(lat_long_1, lat_long_2).kilometers
    return distances

# Calculate distances
distances = calculate_distances(locations)

# Rank distances for each location and add to DataFrame
meta['ranked_site_id'] = meta['site_id'].apply(
    lambda x: [site for site, _ in sorted(distances[x].items(), key=lambda item: item[1])]
)
loc_dict = meta.set_index('site_id')['ranked_site_id'].to_dict()

In [36]:
missing_sites = ['merced_river_yosemite_at_pohono_bridge',
                 'american_river_folsom_lake',
                 'san_joaquin_river_millerton_reservoir']
missing_sites_near = {}
for site in missing_sites:
    nearest_sites = []
    site_lst = loc_dict[site]
    for near_sites in site_lst:
        if len(nearest_sites) == 3:
            break
        if near_sites not in missing_sites:
            nearest_sites.append(near_sites)
    missing_sites_near[site] = nearest_sites
missing_sites_near
    
# filtered_df = data[data['site_id'].isin(missing_sites)][['site_id','volume','pred_apr','pred_may','pred_jun','pred_jul']].drop_duplicates()
# filtered_df

{'merced_river_yosemite_at_pohono_bridge': ['virgin_r_at_virtin',
  'owyhee_r_bl_owyhee_dam',
  'boise_r_nr_boise'],
 'american_river_folsom_lake': ['owyhee_r_bl_owyhee_dam',
  'detroit_lake_inflow',
  'boise_r_nr_boise'],
 'san_joaquin_river_millerton_reservoir': ['virgin_r_at_virtin',
  'owyhee_r_bl_owyhee_dam',
  'boise_r_nr_boise']}

In [43]:
from sklearn.linear_model import LinearRegression
sites = missing_sites_near['merced_river_yosemite_at_pohono_bridge']
filtered_df = data[data['site_id'].isin(sites)]
reshaped_df = pd.DataFrame()
reshaped_df['issue_date'] = filtered_df['issue_date'].unique()
reshaped_df['year'] = site_df['year'].values
features = []
for site_id in sites:
    site_df = filtered_df[filtered_df['site_id'] == site_id]
    for feature in ['pred_apr', 'pred_may', 'pred_jun', 'pred_jul']:
        reshaped_df[f'{site_id}_{feature}'] = site_df[feature].values
        features.append(f'{site_id}_{feature}')
reshaped_df = reshaped_df.drop('issue_date', axis = 1)
specific_site = data[data['site_id'] == 'merced_river_yosemite_at_pohono_bridge']
reshaped_df = reshaped_df.merge(specific_site[['year','volume']],
                                on = 'year',
                                how = 'left').drop_duplicates().reset_index()
# X = sm.add_constant(reshaped_df[features])
X = reshaped_df[features]
y = reshaped_df['volume']
mry_lr_model = LinearRegression(positive=True)
mry_lr_model.fit(X,y)
with open('mry_raw_model.pkl', 'wb') as file:
    pickle.dump(mry_lr_model, file)

In [44]:
sites = missing_sites_near['american_river_folsom_lake']
filtered_df = data[data['site_id'].isin(sites)]
reshaped_df = pd.DataFrame()
reshaped_df['issue_date'] = filtered_df['issue_date'].unique()
reshaped_df['year'] = site_df['year'].values
features = []
for site_id in sites:
    site_df = filtered_df[filtered_df['site_id'] == site_id]
    for feature in ['pred_apr', 'pred_may', 'pred_jun', 'pred_jul']:
        reshaped_df[f'{site_id}_{feature}'] = site_df[feature].values
        features.append(f'{site_id}_{feature}')
reshaped_df = reshaped_df.drop('issue_date', axis = 1)
specific_site = data[data['site_id'] == 'american_river_folsom_lake']
reshaped_df = reshaped_df.merge(specific_site[['year','volume']],
                                on = 'year',
                                how = 'left').drop_duplicates().reset_index()
# X = sm.add_constant(reshaped_df[features])
X = reshaped_df[features]
y = reshaped_df['volume']
arf_lr_model = LinearRegression(positive=True)
arf_lr_model.fit(X,y)
with open('arf_raw_model.pkl', 'wb') as file:
    pickle.dump(arf_lr_model, file)

In [45]:
sites = missing_sites_near['san_joaquin_river_millerton_reservoir']
filtered_df = data[data['site_id'].isin(sites)]
reshaped_df = pd.DataFrame()
reshaped_df['issue_date'] = filtered_df['issue_date'].unique()
reshaped_df['year'] = site_df['year'].values
features = []
for site_id in sites:
    site_df = filtered_df[filtered_df['site_id'] == site_id]
    for feature in ['pred_apr', 'pred_may', 'pred_jun', 'pred_jul']:
        reshaped_df[f'{site_id}_{feature}'] = site_df[feature].values
        features.append(f'{site_id}_{feature}')
reshaped_df = reshaped_df.drop('issue_date', axis = 1)
specific_site = data[data['site_id'] == 'san_joaquin_river_millerton_reservoir']
reshaped_df = reshaped_df.merge(specific_site[['year','volume']],
                                on = 'year',
                                how = 'left').drop_duplicates().reset_index()
# X = sm.add_constant(reshaped_df[features])
X = reshaped_df[features]
y = reshaped_df['volume']
sjm_lr_model = LinearRegression(positive=True)
sjm_lr_model.fit(X,y)
with open('sjm_raw_model.pkl', 'wb') as file:
    pickle.dump(sjm_lr_model, file)